In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import os
import numpy as np

text = "bestcase"

# 定义文件夹路径
data_folder = os.path.join('Data', text)

# 定义文件列表 - Simple Regret来自pre文件，Cumulative Regret来自原始文件
simple_regret_files = ['200_10_smallest_pre.csv','200_10_cluster_pre.csv', '200_10_dynamic_pre.csv', '200_10_cyclic_pre.csv']
cumulative_regret_files = ['200_10_smallest.csv','200_10_cluster.csv','200_10_dynamic.csv', '200_10_cyclic.csv']

curve_names = ['smallest','cluster', 'BCUCB', 'BCUCB+cyclic']
colors = px.colors.qualitative.Plotly[:len(curve_names)] 

# 创建DataFrame存储Simple Regret和Cumulative Regret
simple_regret_data = pd.DataFrame()
cumulative_regret_data = pd.DataFrame()

# 加载Simple Regret数据（来自pre文件）
for file, name in zip(simple_regret_files, curve_names):
    file_path = os.path.join(data_folder, file)
    df = pd.read_csv(file_path, index_col='Repetition')
    
    # 计算每个step的均值（Simple Regret）
    mean_results = df.mean(axis=0)
    
    # 添加到Simple Regret DataFrame
    temp_df = pd.DataFrame({
        'Iter': range(1, 151),
        'Value': mean_results.values,
        'Condition': name
    })
    simple_regret_data = pd.concat([simple_regret_data, temp_df])

# 加载Cumulative Regret数据（来自原始文件）
for file, name in zip(cumulative_regret_files, curve_names):
    file_path = os.path.join(data_folder, file)
    df = pd.read_csv(file_path, index_col='Repetition')
    
    # 计算每个step的均值
    mean_results = df.mean(axis=0)
    
    # 计算Cumulative Regret
    cumsum_results = np.cumsum(mean_results.values)
    temp_df = pd.DataFrame({
        'Iter': range(1, 151),
        'Value': cumsum_results,
        'Condition': name
    })
    cumulative_regret_data = pd.concat([cumulative_regret_data, temp_df])

# 创建子图（2行1列，垂直排列）- 删除子图标题
fig = make_subplots(
    rows=2, cols=1,
    vertical_spacing=0.1
    # 删除了 subplot_titles 参数
)

# 第一子图：Cumulative Regret（来自原始文件）- 先画Cumulative Regret
for i, condition in enumerate(curve_names):
    cumsum_df = cumulative_regret_data[cumulative_regret_data['Condition'] == condition]
    fig.add_trace(
        go.Scatter(
            x=cumsum_df['Iter'],
            y=cumsum_df['Value'],
            name=condition,
            legendgroup=condition,
            showlegend=True,  # 在第一子图显示图例
            line=dict(color=colors[i],width=2.5),
            meta=[condition],
            hovertemplate="Iter: %{x}<br>Cumulative Regret: %{y:.4f}<br>Condition: %{meta[0]}<extra></extra>"
        ),
        row=1, col=1  # Cumulative Regret放在第一行
    )

# 第二子图：Obs-based Simple Regret（来自pre文件）
for i, condition in enumerate(curve_names):
    simple_df = simple_regret_data[simple_regret_data['Condition'] == condition]
    fig.add_trace(
        go.Scatter(
            x=simple_df['Iter'],
            y=simple_df['Value'],
            name=condition,
            legendgroup=condition,
            showlegend=False,  # 避免重复图例
            line=dict(color=colors[i],width=2.5),
            meta=[condition],
            hovertemplate="Iter: %{x}<br>Simple Regret: %{y:.4f}<br>Condition: %{meta[0]}<extra></extra>"
        ),
        row=2, col=1  # Simple Regret放在第二行
    )

# 更新布局
fig.update_layout(
    title_text=text+"_200_10",
    title_x=0.5,
    height=600,
    width=800,
    template='plotly_white',
    legend_title_text='Condition',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.95)
)

# 更新Y轴和X轴标签
fig.update_yaxes(title_text="Cumulative Regret", row=1, col=1)  # 第一子图Y轴标签
fig.update_yaxes(title_text="Simple Regret", row=2, col=1)      # 第二子图Y轴标签
fig.update_xaxes(title_text="Iteration", row=2, col=1)          # 仅底部子图显示X轴标签

# 显示图形
fig.show()

# 保存为HTML文件（可选）
# pio.write_html(fig, 'combined_simple_cumulative_plot.html')
#fig.write_image(text+"_200_10", format="png", width=800, height=600, scale=3)